In [180]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [181]:
# Import Depencies

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import numpy as np
import requests
from pprint import pprint
import time
import json
from requests.structures import CaseInsensitiveDict

from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [182]:
#Section for user input

lat = ''
lon = ''

default = [0, 0]
cleveland = [41.4993, -81.6944]
atlanta = [33.7488, -84.3877]
los_angeles = [34.0549, -118.2426]
new_york_city = [40.7128, -74.0060]
chicago = [41.8781, -87.6298]
seattle = [47.6061, -122.3328]

location_search = [123456789,123456789] #user_defined or written to pre-determined

default_city_list =  {'Cleveland' : cleveland , 'Atlanta' : atlanta, 'Los Angeles' : los_angeles, 'New York City' : new_york_city, 'Chicago' : chicago, 'Seattle' : seattle}


print('Available list of cities\n')
for k, v in default_city_list.items():
    print(k)



Available list of cities

Cleveland
Atlanta
Los Angeles
New York City
Chicago
Seattle


In [183]:
user_coordinates = input("Would you like pick from a list of available cities? yes or no")

if user_coordinates.lower() == 'y' or user_coordinates.lower() == 'yes':

    match = input("Please pick a city from the list.\n")

    for k, v in default_city_list.items():
        if match.lower() == k.lower():
            location_search = v

elif user_coordinates.lower() == 'n' or user_coordinates.lower() == 'no':
    user_lat = input("Please enter the latitude. ")
    user_lon = input("Please enter the longitude. ")
    location_search = [float(user_lat), float(user_lon)]

else:
    raise Exception("Invalid Input. Must specify Yes (y) or No (n)")

if location_search == [123456789, 123456789]:
    raise Exception("Invalid location. Use a name that is in the list, or choose No (n) your own coordinates.")
    
print(f'Using coordinates {location_search}\n')


Using coordinates [41.4993, -81.6944]



In [184]:

#Default values
radius_meters = 5000
country_code = ''
limit = 100
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

#keep for future flexibility
if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [185]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
# any category can be added - it must also be added to category_dict
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [186]:
# dictionary of each category that will be captured
# we also may want to include user input for this too. Loop through these, or other questions. Set value to True. Default value is false. Must have at least one of these active in order
# to run the script. If not, throw an error
# We either need to specify that the list needs to be under 500, because that is the max number, or create 
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# API string built for flexibility
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"


In [187]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json_info = resp.json()
#pprint(json_info) #only for checking data

200


In [188]:
#only for debugging
json_info

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Great Lakes Science Center',
    'country': 'United States',
    'country_code': 'us',
    'state': 'Ohio',
    'county': 'Cuyahoga County',
    'city': 'Cleveland',
    'postcode': '44114',
    'district': 'Downtown Cleveland',
    'street': 'Erieside Avenue',
    'housenumber': '601',
    'lon': -81.69699352039575,
    'lat': 41.507522949999995,
    'state_code': 'OH',
    'formatted': 'Great Lakes Science Center, 601 Erieside Avenue, Cleveland, OH 44114, United States of America',
    'address_line1': 'Great Lakes Science Center',
    'address_line2': '601 Erieside Avenue, Cleveland, OH 44114, United States of America',
    'categories': ['building',
     'building.tourism',
     'entertainment',
     'entertainment.museum',
     'internet_access',
     'wheelchair',
     'wheelchair.yes'],
    'details': ['details',
     'details.building',
     'details.contact',
     'details.facilities',
 

In [189]:
#need error checking to make sure it is only US locations. String split is based on US cities.
if json_info["features"][0]["properties"]["country_code"] != 'us':
    raise Exception('This process is built for US cities only.')


In [190]:
features_pd = pd.json_normalize(json_info["features"])

In [191]:
properties_df = features_pd[["properties.address_line1", "properties.address_line2", "properties.place_id"]]

In [192]:
# Rename columns
properties_df = properties_df.rename(columns = {"properties.address_line1": "Property", "properties.address_line2": "Address2", "properties.place_id": "Place_ID"})
#properties_df

In [193]:
properties_df

,Property,Address2,Place_ID
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
...,...,...,...
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...
98,Kram & Kram Auto Repair,"East 55th Street, Cleveland, OH 44105, United ...",51fe44a40caf6954c05976911f70b3c04440f00102f901...


In [194]:
# Create separate columns for the address fields
properties_df[['Address', 'City', 'State_Zip', 'Country']] = properties_df.Address2.str.split(",", expand = True)
#properties_df

In [195]:
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
...,...,...,...,...,...,...,...
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America
98,Kram & Kram Auto Repair,"East 55th Street, Cleveland, OH 44105, United ...",51fe44a40caf6954c05976911f70b3c04440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America


In [196]:
properties_df = properties_df.dropna()
#properties_df

In [197]:
properties_df["State_Zip"] = properties_df['State_Zip'].str.strip()
#properties_df

In [198]:
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
...,...,...,...,...,...,...,...
94,M&I Deli,"East 55th Street, Cleveland, OH 44105, United ...",512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America


In [199]:
#I don't think we need this anymore
properties_df.State_Zip.str.split(" ", expand = True)

,0,1
0,OH,44114
1,OH,44114
2,OH,44109
3,OH,44113
4,OH,44115
...,...,...
94,OH,44105
95,OH,44105
96,OH,44103
97,OH,44114


In [200]:
properties_df[['State', 'Zip']] = properties_df.State_Zip.str.split(" ", expand = True)
#properties_df

In [201]:
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America,OH,44114
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America,OH,44114
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America,OH,44109
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America,OH,44113
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America,OH,44115
...,...,...,...,...,...,...,...,...,...
94,M&I Deli,"East 55th Street, Cleveland, OH 44105, United ...",512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America,OH,44103
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America,OH,44114


In [202]:
properties_df = properties_df.drop('Address2', axis=1)
#properties_df

In [203]:
properties_df = properties_df.drop('State_Zip', axis=1)
#properties_df

In [204]:
properties_df

,Property,Place_ID,Address,City,Country,State,Zip
0,Great Lakes Science Center,51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,United States of America,OH,44114
1,International Women's Air and Space Museum,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,United States of America,OH,44114
2,A Christmas Story House Museum,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,United States of America,OH,44109
3,Frederich Bicycle,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,United States of America,OH,44113
4,Graffiti Caps,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,United States of America,OH,44115
...,...,...,...,...,...,...,...
94,M&I Deli,512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,United States of America,OH,44105
95,Allen Hardware,515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,United States of America,OH,44105
96,Rite Aid,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,United States of America,OH,44103
97,A-To-Z Appliance and Furniture,51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,United States of America,OH,44114


In [205]:
properties_df = properties_df.reindex(['Place_ID', 'Property', 'Address', 'City', 'State', 'Zip', 'Country'], axis=1)

In [206]:
properties_df

,Place_ID,Property,Address,City,State,Zip,Country
0,51512b54fd976c54c059bad57810f5c04440f00102f901...,Great Lakes Science Center,601 Erieside Avenue,Cleveland,OH,44114,United States of America
1,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,International Women's Air and Space Museum,1501 North Marginal Road,Cleveland,OH,44114,United States of America
2,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,A Christmas Story House Museum,1103 Rowley Avenue,Cleveland,OH,44109,United States of America
3,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,Frederich Bicycle,3800 Lorain Avenue,Cleveland,OH,44113,United States of America
4,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,Graffiti Caps,3200 Carnegie Avenue,Cleveland,OH,44115,United States of America
...,...,...,...,...,...,...,...
94,512940a8d0bb6954c0592857231292bf4440f00102f901...,M&I Deli,East 55th Street,Cleveland,OH,44105,United States of America
95,515706584bac6954c059fc9216f05fbf4440f00102f901...,Allen Hardware,2733 East 55th Street,Cleveland,OH,44105,United States of America
96,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,Rite Aid,East 53rd Street,Cleveland,OH,44103,United States of America
97,51aa379682bd6954c059bf63cda508c14440f00102f901...,A-To-Z Appliance and Furniture,Commerce Avenue,Cleveland,OH,44114,United States of America


In [207]:
#There is something wrong with this. I don't know what
#properties_df = properties_df.drop_duplicates(subset = "Place_ID", inplace = True)


In [212]:
properties_df

,Place_ID,Property,Address,City,State,Zip,Country
0,51512b54fd976c54c059bad57810f5c04440f00102f901...,Great Lakes Science Center,601 Erieside Avenue,Cleveland,OH,44114,United States of America
1,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,International Women's Air and Space Museum,1501 North Marginal Road,Cleveland,OH,44114,United States of America
2,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,A Christmas Story House Museum,1103 Rowley Avenue,Cleveland,OH,44109,United States of America
3,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,Frederich Bicycle,3800 Lorain Avenue,Cleveland,OH,44113,United States of America
4,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,Graffiti Caps,3200 Carnegie Avenue,Cleveland,OH,44115,United States of America
...,...,...,...,...,...,...,...
94,512940a8d0bb6954c0592857231292bf4440f00102f901...,M&I Deli,East 55th Street,Cleveland,OH,44105,United States of America
95,515706584bac6954c059fc9216f05fbf4440f00102f901...,Allen Hardware,2733 East 55th Street,Cleveland,OH,44105,United States of America
96,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,Rite Aid,East 53rd Street,Cleveland,OH,44103,United States of America
97,51aa379682bd6954c059bf63cda508c14440f00102f901...,A-To-Z Appliance and Furniture,Commerce Avenue,Cleveland,OH,44114,United States of America


In [208]:
properties_df

,Place_ID,Property,Address,City,State,Zip,Country
0,51512b54fd976c54c059bad57810f5c04440f00102f901...,Great Lakes Science Center,601 Erieside Avenue,Cleveland,OH,44114,United States of America
1,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,International Women's Air and Space Museum,1501 North Marginal Road,Cleveland,OH,44114,United States of America
2,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,A Christmas Story House Museum,1103 Rowley Avenue,Cleveland,OH,44109,United States of America
3,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,Frederich Bicycle,3800 Lorain Avenue,Cleveland,OH,44113,United States of America
4,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,Graffiti Caps,3200 Carnegie Avenue,Cleveland,OH,44115,United States of America
...,...,...,...,...,...,...,...
94,512940a8d0bb6954c0592857231292bf4440f00102f901...,M&I Deli,East 55th Street,Cleveland,OH,44105,United States of America
95,515706584bac6954c059fc9216f05fbf4440f00102f901...,Allen Hardware,2733 East 55th Street,Cleveland,OH,44105,United States of America
96,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,Rite Aid,East 53rd Street,Cleveland,OH,44103,United States of America
97,51aa379682bd6954c059bf63cda508c14440f00102f901...,A-To-Z Appliance and Furniture,Commerce Avenue,Cleveland,OH,44114,United States of America


In [213]:
#Convert pandas dataframe to polars
properties_df_pl = pl.from_pandas(properties_df)

In [214]:
properties_df_pl

Place_ID,Property,Address,City,State,Zip,Country
str,str,str,str,str,str,str
"""51512b54fd976c…","""Great Lakes Sc…","""601 Erieside A…",""" Cleveland""","""OH""","""44114""",""" United States…"
"""51b2dc88dd2d6c…","""International …","""1501 North Mar…",""" Cleveland""","""OH""","""44114""",""" United States…"
"""5164ffa097066c…","""A Christmas St…","""1103 Rowley Av…",""" Cleveland""","""OH""","""44109""",""" United States…"
"""5118963fdf966d…","""Frederich Bicy…","""3800 Lorain Av…",""" Cleveland""","""OH""","""44113""",""" United States…"
"""5151fb08579b6a…","""Graffiti Caps""","""3200 Carnegie …",""" Cleveland""","""OH""","""44115""",""" United States…"
"""51fa4800ed7d6c…","""Leavened""","""1633 Auburn Av…",""" Cleveland""","""OH""","""44113""",""" United States…"
"""514459f8fada6f…","""Christophier C…","""9308 Clifton B…",""" Cleveland""","""OH""","""44102""",""" United States…"
"""5166b418a7826c…","""Rock and Roll …","""East 9th Stree…",""" Cleveland""","""OH""","""44114""",""" United States…"
"""516249b9fb1c6c…","""Cleveland Hung…","""Selzer Way""",""" Cleveland""","""OH""","""44114""",""" United States…"


In [209]:
#create empty categories dataframe
category_table_df = pd.DataFrame(columns = ['Place_ID', 'Category'])

In [210]:
#insert each row into categories dataframe
num_of_records = len(json_info['features'])

for i in range(num_of_records):
    place_id_var = json_info["features"][i]["properties"]["place_id"]

    for category in json_info["features"][i]["properties"]["categories"]:
        #print(j)
        new_row = {'Place_ID': place_id_var, 'Category': category}

        #https://stackoverflow.com/questions/75956209/error-dataframe-object-has-no-attribute-append
        category_table_df = category_table_df._append(new_row, ignore_index = True)

In [211]:
#convert to polaris dataframe
category_table_df_pl = pl.from_pandas(category_table_df)
category_table_df_pl

Place_ID,Category
str,str
"""51512b54fd976c…","""building"""
"""51512b54fd976c…","""building.touri…"
"""51512b54fd976c…","""entertainment"""
"""51512b54fd976c…","""entertainment.…"
"""51512b54fd976c…","""internet_acces…"
"""51512b54fd976c…","""wheelchair"""
"""51512b54fd976c…","""wheelchair.yes…"
"""51b2dc88dd2d6c…","""entertainment"""
"""51b2dc88dd2d6c…","""entertainment.…"
